# Audio-Filters
## Characteristics in Transfer Function and Impulse Response#
### Modules

In [ ]:
%load_ext autoreload
%autoreload 2

import matplotlib
import matplotlib.pyplot as plt
import numpy as np
from scipy import signal
import audiofilter  # from https://github.com/spatialaudio/digital-signal-processing-lecture
%matplotlib inline
matplotlib.rcParams['font.size'] = 8
fsz = (16*0.7, 9*0.7)  #fig size

fs = 48000  # sampling frequency in Hz
Nt = 2**12

def impz(b=1, a=1, Nt=2**12, fs=48000):
    t = np.arange(0,Nt) / fs
    dirac = np.zeros(Nt)
    dirac[0] = 1
    return t, signal.lfilter(b,a,dirac)

def magnitude_plot(x, y, title, legend):
    fig = plt.figure(figsize=fsz)
    audiofilter.magnitude_plot_overlay(x, y, title, legend, fig=fig) 

def plot_system(ttl):
    t, h = impz(b, a, Nt, fs)
    W, H = signal.freqz(b, a, Nt)
    W, gdly = signal.group_delay((b, a), W)

    plt.figure(figsize=fsz)
    plt.subplot(2,2,1)
    plt.semilogx(W/np.pi*fs/2,20*np.log10(np.abs(H)))
    plt.xlim(10,20000)
    plt.grid()
    plt.xlabel("f / Hz")
    plt.ylabel("A / dB")
    plt.title("Magnitude (Level): "+ ttl)
    plt.ylim(-40,10)

    plt.subplot(2,2,2)
    plt.semilogx(W/np.pi*fs/2,(np.angle(H))*180/np.pi)
    #plt.semilogx(W/np.pi*fs/2,np.unwrap(np.angle(H))*180/np.pi)    
    #plt.plot(W/np.pi*fs/2,np.unwrap(np.angle(H))*180/np.pi)
    plt.xlim(10,20000)
    #plt.ylim(-180, 180)
    plt.grid()
    plt.xlabel("f / Hz")
    plt.ylabel(r"$\phi$ / deg")
    plt.title("Phase")

    plt.subplot(2,2,3)
    plt.plot(t*1000,h)
    plt.xlim(-1,10)
    plt.grid()
    plt.xlabel("t / ms")
    plt.ylabel("h(t)")
    plt.title("Impulse Response")

    plt.subplot(2,2,4)
    plt.semilogx(W/np.pi*fs/2,gdly/fs*1000)
    plt.xlim(10,20000)
    plt.ylim(-2, 6)
    plt.grid()
    plt.xlabel("f / Hz")
    plt.ylabel("t / ms")
    plt.title("Group Delay")


### Basics

In [ ]:
b = np.zeros(Nt)
b[0] = 10**(-10/20)
a = 1
plot_system("Fader Gain")
print(b[0])

In [ ]:
b = np.zeros(Nt)
b[0] = 10**(+3.01/20)
a = 1
plot_system("Fader Gain")
print(b[0])

In [ ]:
b = np.zeros(Nt)
b[0] = 0
b[int(np.ceil(1/1000*fs))] = 1
a = 1
plot_system("Delay")

### Combfilter

In [ ]:
# combfilter FIR, 1.5ms delay, -16dB ->
# max stereo cues for delay and magnitude difference between left / right
b = np.zeros(200)
b[0] = 1
b[int(np.ceil(1.5/1000*fs))] = 10**(-16/20)
a = 1
plot_system("Combfilter")

### Infinite Impulse Response (IIR) Filters

In [ ]:
fc = 100  # Hz
B, A, b, a = audiofilter.biquad_lp2nd(fc, fs)
plot_system("Butterworth Lowpass Filter 2nd order")

In [ ]:
BW = 2  # bandwidth in octaves
fm = 2000  # mid frequency in Hz; using fm=1000 and BW=2, then fc is at 1 kHz
G = -18  # dB
QBP = audiofilter.q_from_bw(BW)
B1, A1, b, a = audiofilter.biquad_peq2nd(fm, G, QBP, fs, "I")  # fc at -3dB
B2, A2, b, a = audiofilter.biquad_peq2nd(fm, G, QBP, fs, "II")  # fc at -15dB
B3, A3, b, a = audiofilter.biquad_peq2nd(fm, G, QBP, fs, "III")  # fc at -9dB
f = np.arange(1,fs/2,1)
s = 2*np.pi*f
s, H1 = signal.freqs(B1, A1, s)
s, H2 = signal.freqs(B2, A2, s)
s, H3 = signal.freqs(B3, A3, s)

x = np.column_stack((f, f, f))
y = np.column_stack((H1, H2, H3))
title = 'PEQ 2nd order for cutting'
legend = ["type I", "type II", "type III"]
magnitude_plot(x, y, title, legend)

In [ ]:
fc = 1000  # Hz
G = 18  # dB
B1, A1, b, a = audiofilter.biquad_hshv2nd(fc, G, fs, "I")  # fc at 15 dB
B2, A2, b, a = audiofilter.biquad_hshv2nd(fc, G, fs, "II")  # fc at 3 dB
B3, A3, b, a = audiofilter.biquad_hshv2nd(fc, G, fs, "III")  # fc at 9 dB
f = np.arange(1,fs/2,1)
s = 2*np.pi*f
s, H1 = signal.freqs(B1, A1, s)
s, H2 = signal.freqs(B2, A2, s)
s, H3 = signal.freqs(B3, A3, s)

x = np.column_stack((f, f, f))
y = np.column_stack((H1, H2, H3))
title = 'High-Shelve 2nd order'
legend = ["type I", "type II", "type III"]
magnitude_plot(x, y, title, legend)

### Linear Phase Finite Impulse Response (FIR) Filters

In [ ]:
NFIR = 481  #481 => 0.5 ms group delay at fs=48 kHz
b = np.random.randn(NFIR) / 40
tmp = b[0:int((NFIR-1)/2)]
b = np.concatenate((tmp[::], b[int((NFIR-1)/2)], +tmp[::-1]), axis=None)
a = 1
plot_system("Linear Phase FIR Type I, axial-symm, odd coeff")

In [ ]:
NFIR = 481  #0.5 ms group delay at fs=48 kHz
b = np.random.randn(NFIR) / 40
b = np.concatenate((tmp[::], 0, -tmp[::-1]), axis=None)
a = 1
plot_system("Linear Phase FIR Type III, point-symm, odd coeff")

In [ ]:
NFIR = 480  #0.5 ms group delay at fs=48 kHz
b = np.random.randn(NFIR) / 40
tmp = b[0:int(NFIR/2)]
b = np.concatenate((tmp[::], +tmp[::-1]), axis=None)
a = 1
plot_system("Linear Phase FIR Type II, axial-symm, even coeff")

In [ ]:
NFIR = 480  #0.5 ms group delay at fs=48 kHz
b = np.random.randn(NFIR) / 40
tmp = b[0:int(NFIR/2)]
b = np.concatenate((tmp[::], -tmp[::-1]), axis=None)
a = 1
plot_system("Linear Phase FIR Type IV, point-symm, even coeff")

In [ ]:
NFIR = 481  #0.5 ms group delay at fs=48 kHz
n = np.arange(0,NFIR) 
b = np.sin(np.pi/4*n) / (np.pi*n)  # fcut = 6 kHz at fs=48 kHz
b[0] = 1/4  # L'Hospital rule
tmp = b[1:int((NFIR-1)/2)+1]
b = np.concatenate((tmp[::-1], b[0], +tmp[::]), axis=None)
a = 1
plot_system("rect win lowpass , Linear Phase FIR Type I, axial-symm, odd coeff")
